# LOWER BOUND MRS

## PREPARING THE ENVIRONMENT

Installing the relevant libraries:

In [1]:
import os
import mip
import numpy as np
import pandas as pd

Defining the constants:

In [2]:
BIN_SIZE = [100,
            120,
            150]

MAX_COST = BIN_SIZE[-1]

Defining classes and functions:

In [3]:
class Instance:
    def __init__(self, filename):
        self.n = None
        self.v = None
        self.G = None

        with open(filename, 'r') as file:
            self.n = int(file.readline().strip())
            self.v = np.zeros(self.n, dtype=int)
            self.G = np.zeros((self.n, self.n), dtype=int)

            for line in file:
                tokens = list(map(int, line.split()))
                if not tokens:
                    continue

                v1 = tokens[0]
                self.v[v1] = tokens[1]

                for v2 in tokens[2:]:
                    self.G[v1][v2] = 1
                    self.G[v2][v1] = 1

        for v1 in range(self.n - 1):
            for v2 in range(v1 + 1, self.n):
                if self.v[v1] + self.v[v2] > MAX_COST:
                    self.G[v1][v2] = 1
                    self.G[v2][v1] = 1

    def __getitem__(self, i):
        return self.v[i]

    def describe(self):
        print(f'Number of items = {self.n}')

        print('v =')
        print(self.v)
        print('G =')
        print(self.G)

Reading an instance:

In [4]:
filename = '../instances/train/Correia_Random_2_3_2_5.txt'

instance = Instance(filename)

instance.describe()

Number of items = 200
v =
[ 98  82  79  61  95  88  89  78  60  69  92  61  62  93  83  50  66  80
  72  67  87  79  78  53  76  70  52  87  86  79  70  86  51  92  60 100
  86  60  74  91  93  94  55  94  77  50  65  84  70  61  77  70  85  79
  83 100  78  77  99  91  90  65  84  78  76  72  84  97  58  80  52  74
  54  51  54  68  76  87  73  60  92  54  58  57  93  71 100  97  77  96
  99  57  67  93  90  92  87  89  69  51  71  77  92  93  66  96  92  89
  53  80  84  55  65  65  62  50  52  93  81  93  88  97  57  92  69  69
  80  63  89  87  89  60 100  93  83  82  86  62  70  57  69  65  55  64
  66  81  88  70  69  86  58  59  62  81  79  92  62  58  74 100  77  98
  66  65  82  74  96  95  61  70  96  71  95  85  68  54  58  73  83  80
  63  53  90  85  95  85  69  50  76  82  67  75  84  82  77  53  70  77
  76  58]
G =
[[0 1 1 ... 1 1 1]
 [1 0 1 ... 1 1 0]
 [1 1 0 ... 1 1 0]
 ...
 [1 1 1 ... 0 1 0]
 [1 1 1 ... 1 0 0]
 [1 0 0 ... 0 0 0]]


Loading heuristic upper bounds:

In [5]:
data_train = pd.read_csv('../out/results_train_final.txt', sep='\s+')
data_test  = pd.read_csv('../out/results_test_final.txt' , sep='\s+')

data = pd.concat([data_train, data_test])

instance2heuristic = dict(zip(data['instance'], data['obj_lnsa']))

## GETTING A MAXIMUM CLIQUE

Defining a function to obtain a maximal clique using the algorithm described by Johnson:

In [6]:
def johnson_maximal_clique(instance):
    S = set()
    R = set(range(instance.n))

    while R:
        y = max(R, key=lambda v: sum(instance.G[v][w] for w in R))

        S.add(y)
        R = {w for w in R if instance.G[y][w] == 1}

    return S

def verify_maximal_clique(clique):
    for v in clique:
        for w in clique:
            if v != w and not instance.G[v][w]:
                return False

    return True

Getting a maximum clique:

In [7]:
clique = johnson_maximal_clique(instance)

print('Maximum clique =', clique)

Maximum clique = {0, 1, 2, 4, 5, 6, 7, 10, 13, 14, 17, 20, 21, 22, 24, 27, 28, 29, 31, 33, 35, 36, 39, 40, 41, 43, 44, 47, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 66, 67, 69, 76, 77, 80, 84, 86, 87, 88, 89, 90, 93, 94, 95, 96, 97, 101, 102, 103, 105, 106, 107, 109, 110, 117, 118, 119, 120, 121, 123, 126, 128, 129, 130, 132, 133, 134, 135, 136, 145, 146, 149, 153, 154, 155, 159, 160, 161, 164, 166, 167, 170, 172, 173, 178, 179, 182, 183, 184, 185, 188, 189, 191, 192, 193, 194, 197, 198}


Checking for maximal clique:

In [8]:
if verify_maximal_clique(clique):
    print('The maximum clique obtained is valid.')
else:
    print('The maximum clique obtained is not valid.')

The maximum clique obtained is valid.


## LOWER BOUND

Getting relevant information for models:

In [9]:
def get_params(instance, clique, bins):
    n = instance.n
    V = instance.v
    G = instance.G

    B = list(range(len(bins)))

    Vc  = clique
    Vc_ = set(range(n)) - clique

    Vck = {k : [] for k, _ in enumerate(B)}
    for i in Vc:
        vi = V[i]

        k  = 0
        while vi > BIN_SIZE[k]:
            k += 1

        Vck[k].append(i)

    Vmin = {
        k : min(V[i] for i in vck)
        for k, vck in Vck.items()
        if vck
    }

    return n, V, G, B, Vc, Vc_, Vck, Vmin


n, V, G, B, Vc, Vc_, Vck, Vmin = get_params(instance, clique, BIN_SIZE)

### CALCULATING W(j, k)

Defining a function to calculate W(j, k):

In [10]:
def w_jk(instance, clique_comp, Wj, v_min):
    n = instance.n
    V = instance.v
    G = instance.G

    model = mip.Model(sense=mip.MAXIMIZE)
    model.verbose = 0

    x = {i: model.add_var(var_type=mip.BINARY) for i in clique_comp}

    model.objective = mip.xsum(V[i] * x[i] for i in x)

    model += model.objective <= Wj - v_min

    status = model.optimize()
    obj    = model.objective_value

    return status, int(obj)

Calculating the effective space for each bin type:

In [11]:
W = dict()

for k, vck in Vck.items():
    if vck:
        vmin = Vmin[k]

        for j in range(k, len(B)):
            status, obj = w_jk(instance, Vc_, BIN_SIZE[j], vmin)

            if status == mip.OptimizationStatus.OPTIMAL:
                W[j, k] = obj
            else:
                print(f'Infeasibility in type bin {j} and v{k}!')

for (j, k), w in W.items():
    print(f'Type bin {j} and v{k} =', w)

Type bin 0 and v0 = 0
Type bin 1 and v0 = 0
Type bin 2 and v0 = 74


### LOWER BOUND

Defining the integer linear programming model to the lower bound:

In [12]:
def lower_bound(instance, B, Vc_, Vck, BIN_SIZE, W, time_limit=120, heuristic=None):
    n = instance.n
    V = instance.v
    G = instance.G

    model = mip.Model(sense=mip.MINIMIZE)
    model.verbose = 0

    y = {k      : model.add_var(var_type=mip.INTEGER, lb=0) for k in B}
    t = {(j, k) : model.add_var(var_type=mip.BINARY) for (j, k) in W}

    model.objective = (
        mip.xsum(BIN_SIZE[k] * y[k] for k in B) +
        mip.xsum(BIN_SIZE[j] * t[j, k] for (j, k) in t)
    )

    if heuristic:
        model += model.objective <= heuristic

    for k, vck in Vck.items():
        if vck:
            model += mip.xsum(t[j, k] for j in range(k, len(B))) <= len(vck)

    model += mip.xsum(BIN_SIZE[k] * y[k] for k in range(len(B))) +  \
             mip.xsum(BIN_SIZE[k] * t[j, k] for (j, k) in t)     >= \
             mip.xsum(V[i] for i in range(n))

    model += mip.xsum(BIN_SIZE[k] * y[k] for k in range(len(B))) +  \
             mip.xsum(W[j, k] * t[j, k] for (j, k) in t)         >= \
             mip.xsum(V[i] for i in Vc_)

    status = model.optimize(max_seconds=time_limit)
    obj    = model.objective_value

    del model

    return status, int(obj)

In [13]:
h = instance2heuristic[filename.lstrip('../')]


status, obj = lower_bound(instance, B, Vc_, Vck, BIN_SIZE, W, heuristic=h)

print('Objetive value =', obj)

Objetive value = 15180


## APPLYING TO ALL INSTANCES

Getting the paths to all instances:

In [14]:
filename1 = '../instances/train'
filename2 = '../instances/test'


files = [os.path.join(filename1, f) for f in os.listdir(filename1)] + \
        [os.path.join(filename2, f) for f in os.listdir(filename2)]

print('Number of instances =', len(files))

Number of instances = 720


Getting the lower bounds of all instances:

In [15]:
out = open('../out/lower_bounds.txt', 'a')

sum_gap = 0

for idx, filename in enumerate(files):
    print(f"{idx + 1}. {filename.split('/')[-1].split('.')[0]}", end=' ')

    instance = Instance(filename)

    clique = johnson_maximal_clique(instance)
    n, V, G, B, Vc, Vc_, Vck, Vmin = get_params(instance, clique, BIN_SIZE)

    # Calculating the effective space for each pair of item in the clique and bin type
    W = dict()

    for k, vck in Vck.items():
        if vck:
            vmin = Vmin[k]

            for j in range(k, len(B)):
                status, obj = w_jk(instance, Vc_, BIN_SIZE[j], vmin)

                if status == mip.OptimizationStatus.OPTIMAL:
                    W[j, k] = obj
                else:
                    print(f'Infeasibility in type bin {j} and v{k}!')

    # Lower bound
    filename = filename.lstrip('../')

    h = instance2heuristic[filename]
    status, obj = lower_bound(instance, B, Vc_, Vck,    \
                              BIN_SIZE, W, heuristic=h)

    gap = (h - obj) / obj * 100
    sum_gap += gap

    print(f'{obj} - {h} ({gap:.2f})')
    out.write(f'{filename} {obj}\n')

    del instance, n, V, G, B, \
        Vc, Vc_, Vck, Vmin, W

out.close()

1. Correia_Random_2_3_1_7 15070 - 15350 (1.86)
2. Correia_Random_2_1_1_8 10180 - 10180 (0.00)
3. Correia_Random_2_2_9_3 12200 - 12510 (2.54)
4. Correia_Random_1_2_4_9 6060 - 6080 (0.33)
5. Correia_Random_1_2_0_9 6060 - 6070 (0.17)
6. Correia_Random_2_3_0_6 15510 - 15950 (2.84)
7. Correia_Random_1_1_5_9 4840 - 4850 (0.21)
8. Correia_Random_1_1_6_8 5290 - 5320 (0.57)
9. Correia_Random_1_3_4_6 7690 - 7950 (3.38)
10. Correia_Random_2_2_7_1 12130 - 12230 (0.82)
11. Correia_Random_2_1_3_7 10060 - 10060 (0.00)
12. Correia_Random_2_1_6_1 9570 - 9590 (0.21)
13. Correia_Random_1_2_5_7 6060 - 6090 (0.50)
14. Correia_Random_1_3_4_8 7480 - 7590 (1.47)
15. Correia_Random_2_1_2_6 9730 - 9730 (0.00)
16. Correia_Random_2_2_4_4 12200 - 12210 (0.08)
17. Correia_Random_2_3_0_10 14870 - 15020 (1.01)
18. Correia_Random_2_2_10_1 12130 - 12640 (4.20)
19. Correia_Random_1_1_3_3 4900 - 4910 (0.20)
20. Correia_Random_2_2_4_6 12410 - 12440 (0.24)
21. Correia_Random_1_2_1_4 6240 - 6250 (0.16)
22. Correia_Random_2_

In [16]:
print('Average gap =', sum_gap / len(files))

Average gap = 4.456252795732544
